In [30]:
%load_ext autoreload
%autoreload 2

from nlp_utils import PreProcessing
from nlp_utils import TopicModeling
from google.cloud import bigquery
import pandas as pd
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
nome_da_aba = "Base 34K"
df = pd.read_excel('data/data.xlsx', sheet_name=nome_da_aba)[["video_id","channel_id","video_title", "Label"]]

novo_nome_colunas = [coluna.replace(' ', '_').lower() for coluna in df.columns]
df['video_title'] = df['video_title'].astype(str)
df.columns = novo_nome_colunas

df.head()

,video_id,channel_id,video_title,label
0,-GjJTsTHL_U,UC-8Uff7i2h5qtIvjXJDJqYA,"Arranjos Florais: 19/08, na Fnac da Paulista (SP)",Gardening
1,-RM0Em8oSd8,UC-8Uff7i2h5qtIvjXJDJqYA,Saiba tirar muda da orquídea Phalaenopsis (kei...,Gardening
2,0ORo9kvtOrE,UC-8Uff7i2h5qtIvjXJDJqYA,Regador com garrafa PET e outros truques de ja...,Gardening
3,10L1DZTP1Ak,UC-8Uff7i2h5qtIvjXJDJqYA,Curso de Agricultura Biodinâmica (Fazendeira p...,Gardening
4,1dnkMo7vlBY,UC-8Uff7i2h5qtIvjXJDJqYA,11 plantas perfeitas para ter em local sem sol,Gardening


In [8]:
#NLP
stop_word_list =   ["aqui", "gente", "oi", "então", "ai", "aí", "ufa", "coisa", "[Aplausos]", "[Música]", "né","tá", "vídeo","ne", "ta", "ha", "one", "at",
                    "casar", "coisa","canal", "tipo", "vezes", "mano", "meio", "ea", "eo", "só", "ae", "eis", "caraca", "caramba", "feat", "ft",
                    "graciele_lacerda",  "flavia_calino", "richard_rasmussen", "rezende_evil", "divo_depressao", "receitar_cris", "luisa_mell",
                    "leo_stronda", "divo_depressao", "estevar_mundo", "sal_flor", "tiago_fonseca", "graciele_lacerda", "organizar_frescura", "eduardo_bueno",
                    "debora_aladim", "paula_stephania", "dani_noce	", "tpm_ju", "prof_paulo", "meteoro_doc", "mariano_saad", "familia_brancoala", "osf_rafar",
                    "oliveira", "isabella_fiorentino", "pit_money", "luca_pit", "murilo_coutar", "bianca_andrade", "manual_mundo", "receitar_pai", "dani_noce", "thiago_venturo",
                    "danilo_gentili", "tato_fersoza", "teodoro_tato", "of_the", "like", "it", "house", "danilo"
                    ] #lista de stop words extendida

pp = PreProcessing.PreProcessing(stop_words_list=stop_word_list)

15:35:46 - Adicionanado novas stopwords.
15:35:46 -    Serão adcionada(s) 76 palavra(s)
15:35:46 -    Tamanho da lista de stopwords: 481
15:35:46 -    Tamanho da lista de stopwords: 481


In [26]:
stop_words_custom  =   ["aqui", "gente", "oi", "então", "ai", "aí", "ufa", "coisa", "[Aplausos]", "[Música]", "né","tá", "vídeo","ne", "ta", "ha", "one", "at",
                    "casar", "coisa","canal", "tipo", "vezes", "mano", "meio", "ea", "eo", "só", "ae", "eis", "caraca", "caramba", "feat", "ft",
                    "graciele_lacerda",  "flavia_calino", "richard_rasmussen", "rezende_evil", "divo_depressao", "receitar_cris", "luisa_mell",
                    "leo_stronda", "divo_depressao", "estevar_mundo", "sal_flor", "tiago_fonseca", "graciele_lacerda", "organizar_frescura", "eduardo_bueno",
                    "debora_aladim", "paula_stephania", "dani_noce	", "tpm_ju", "prof_paulo", "meteoro_doc", "mariano_saad", "familia_brancoala", "osf_rafar",
                    "oliveira", "isabella_fiorentino", "pit_money", "luca_pit", "murilo_coutar", "bianca_andrade", "manual_mundo", "receitar_pai", "dani_noce", "thiago_venturo",
                    "danilo_gentili", "tato_fersoza", "teodoro_tato", "of_the", "like", "it", "house", "danilo", "tatá", "fersoza" 
                    ] #lista de stop words extendida


In [31]:
def preprocess_text(text):
    # Adicione as stop words personalizadas à lista de stop words padrão
    stop_words = set(stopwords.words('portuguese'))  # Pode mudar para o idioma desejado
    stop_words.update(stop_words_custom)
    
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Aplica a função de pré-processamento à coluna 'texto'
df['texto_processado'] = df['video_title'].apply(preprocess_text)

# Calcula o TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['texto_processado'])

# Obtém os índices das palavras mais relevantes para cada documento
num_palavras_relevantes = 10  # Altere para o número desejado de palavras relevantes
palavras_relevantes = []

for i in range(len(df)):
    doc = df['texto_processado'].iloc[i]
    tfidf_scores = tfidf_matrix[i, :].toarray()[0]
    top_indices = tfidf_scores.argsort()[-num_palavras_relevantes:][::-1]
    top_words = [tfidf_vectorizer.get_feature_names_out()[idx] for idx in top_indices]
    palavras_relevantes.append(top_words)

# Adiciona a lista de palavras relevantes ao DataFrame
df['palavras_relevantes'] = palavras_relevantes

# Exibe o DataFrame com as palavras relevantes
print(df[['video_title', 'palavras_relevantes']])

                                             video_title  \
0      Arranjos Florais: 19/08, na Fnac da Paulista (SP)   
1      Saiba tirar muda da orquídea Phalaenopsis (kei...   
2      Regador com garrafa PET e outros truques de ja...   
3      Curso de Agricultura Biodinâmica (Fazendeira p...   
4         11 plantas perfeitas para ter em local sem sol   
...                                                  ...   
34558                    Mais um a caminho! Tatá Fersoza   
34559                   Respondendo Vocês - Tatá Fersoza   
34560                    Imagina com dois - Tatá Fersoza   
34561    Meus primeiros 15 dias como mamãe! Tatá Fersoza   
34562  &quot;As perguntas q a gente não aguenta mais ...   

                                     palavras_relevantes  
0      [florais, fnac, arranjos, paulista, sp, 19, 08...  
1      [keikis, phalaenopsis, orquídea, muda, tirar, ...  
2      [regador, malucos, garrafa, jardinagem, costa,...  
3      [biodinâmica, agricultura, fazendeir

In [10]:
start_time = time.time()
# lista_documentos = df_video_transcriptions["transcription_pt"].to_list()
lista_documentos = df["video_title"].to_list()

allowed_postags = ["NOUN","VERB", "PROPN"] # filtro de postags
min_count=5  # Ngram - confiança
threshold=10 # Ngram - suporte
grau=4       # Ngram - quadrigrams
token_to_lower=True #Converter tudo para minusculo 
token_acentuacao=False #Manter Acentuação

corpus, modelo_ngram = pp.pipeline_pre_processamento(lista_documentos, 
                                allowed_postags=allowed_postags,
                                token_to_lower=token_to_lower, 
                                token_acentuacao=token_acentuacao,
                                n_gram_grau=grau,
                                n_gram_min_count=min_count,
                                n_gram_threshold=threshold,
                                stop_word_list=stop_word_list,
                                )
corpus = pp.remover_stop_words(corpus)

print("-> %s Segundos" % (time.time() - start_time))

15:38:28 - Iniciando pipeline de pré processamento:
15:38:28 - Removendo Urls:
15:38:28 - Filtrando Pos Tags
15:40:31 - Tokenizando
15:40:32 - Removendo Stop Words
15:40:32 -     -> Expandindo Stop Words
15:40:32 - Adicionanado novas stopwords.
15:40:32 -    Serão adcionada(s) 76 palavra(s)
15:40:32 -    Tamanho da lista de stopwords: 481
15:40:32 -    Tamanho da lista de stopwords: 481
15:40:33 - Lematizando
15:42:00 - Ngrams: Extraindo os Ngrams a partir de um modelo pré treinado.
15:42:02 - Filtrando tokens menores que 2
-> 215.2613706588745 Segundos


In [14]:
df["corpus"] = corpus
df.head()

,video_id,channel_id,video_title,label,corpus
0,-GjJTsTHL_U,UC-8Uff7i2h5qtIvjXJDJqYA,"Arranjos Florais: 19/08, na Fnac da Paulista (SP)",Gardening,"[arranjo, Fnac, paulista, sp]"
1,-RM0Em8oSd8,UC-8Uff7i2h5qtIvjXJDJqYA,Saiba tirar muda da orquídea Phalaenopsis (kei...,Gardening,"[tirar, mudar, orquidea, phalaenopsis]"
2,0ORo9kvtOrE,UC-8Uff7i2h5qtIvjXJDJqYA,Regador com garrafa PET e outros truques de ja...,Gardening,"[regador, garrafar, pet, truque, jardinagem_ca..."
3,10L1DZTP1Ak,UC-8Uff7i2h5qtIvjXJDJqYA,Curso de Agricultura Biodinâmica (Fazendeira p...,Gardening,"[curso, agricultura, biodinamico, fazendeiro_s..."
4,1dnkMo7vlBY,UC-8Uff7i2h5qtIvjXJDJqYA,11 plantas perfeitas para ter em local sem sol,Gardening,"[planta, sol]"


In [21]:
def tokens_to_text(tokens):
    if tokens is None:
        return ""
    return ' '.join(map(str, tokens))

# Converter cada lista de tokens em documentos de texto, usando a função auxiliar
df['texto'] = df['corpus'].apply(tokens_to_text)

# Criar um objeto TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Ajustar o vetorizador TF-IDF aos documentos de texto
tfidf_matrix = tfidf_vectorizer.fit_transform(df['texto'])

# Obter a matriz TF-IDF em formato de matriz NumPy (opcional)
tfidf_matrix_array = tfidf_matrix.toarray()

# A matriz TF-IDF contém as pontuações TF-IDF para cada termo em cada documento
# Você pode acessar os valores individualmente, se necessário
print(tfidf_matrix_array)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)

# Ajustar o vetorizador TF-IDF aos documentos de texto
tfidf_matrix = tfidf_vectorizer.fit_transform(df['texto'])

# Função para obter os tokens mais relevantes com base nas pontuações TF-IDF
def get_top_tokens(tfidf_matrix, tfidf_vectorizer, num_top_tokens=5):
    top_tokens_per_document = []
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    for i in range(tfidf_matrix.shape[0]):
        tfidf_scores = tfidf_matrix[i].toarray()[0]
        top_indices = tfidf_scores.argsort()[-num_top_tokens:][::-1]
        top_tokens = [feature_names[idx] for idx in top_indices]
        top_tokens_per_document.append(top_tokens)
    
    return top_tokens_per_document

# Chamando a função para obter os tokens mais relevantes
num_top_tokens = 5  # Número de tokens mais relevantes para retornar por documento
top_tokens = get_top_tokens(tfidf_matrix, tfidf_vectorizer, num_top_tokens)

# Adicionando a lista de tokens mais relevantes ao DataFrame 'df'
df['top_tokens'] = top_tokens

# Exibir o DataFrame com os tokens mais relevantes
print(df[['corpus', 'top_tokens']])

NameError: name 'stop_words' is not defined